# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Documentation
__gmaps__: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#heatmaps

__Google places, nearby search request__:  https://developers.google.com/places/web-service/search?hl=es-419#PlaceSearchRequests

__requests module__: https://es.python-requests.org/es/latest/user/quickstart.html

__citipy__: https://github.com/wingchen/citipy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('gmaps_api_key')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather = pd.read_csv("cities_weather_check.csv")
cities_weather.head()

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (°C),Wind Speed (km-h)
0,Mataura,91,NZ,1588641815,85,-46.19,168.86,6.69,15.4
1,Ushuaia,5,AR,1588641657,56,-54.80,-68.30,7.00,40.7
2,Dombarovskiy,0,RU,1588641816,48,50.76,59.54,9.05,9.6
3,Talnakh,1,RU,1588641816,85,69.49,88.40,-11.00,36.0
4,Hong Kong,40,HK,1588641805,70,22.29,114.16,29.00,18.4


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#gmaps configuration
gmaps.configure(api_key= api_key)

#Parameters for map layout
tunisia = (33.791851, 9.520398)
fig_layout = {
    'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '5px'}

#Map
fig = gmaps.figure(map_type='HYBRID',
                  center = tunisia,
                  zoom_level = 2,
                  layout = fig_layout)

#Parameters for heatlayer
coords = cities_weather[["Latitude", "Longitude"]]
weight_hum = cities_weather["Humidity (%)"]

#Heatlayer
heat_layer = gmaps.heatmap_layer(
    coords,
    weights = weight_hum,
    dissipating = True,
    max_intensity = 100,
    point_radius= 8
)

#Add heatlayer to map
fig.add_layer(heat_layer)

#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='5px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_weather = (cities_weather.loc[(cities_weather["Cloudiness (%)"] < 20) &
                                      (cities_weather["Humidity (%)"] < 55) &
                                     (cities_weather["Max Temp (°C)"] < 28)])
print(len(perfect_weather))
perfect_weather.head()

52


,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (°C),Wind Speed (km-h)
2,Dombarovskiy,0,RU,1588641816,48,50.76,59.54,9.05,9.6
35,Makkah al Mukarramah,1,SA,1588641837,26,21.43,39.83,24.99,4.8
40,Albany,3,US,1588641840,52,42.60,-73.97,8.89,16.5
41,Husavik,4,IS,1588641840,45,66.04,-17.34,9.00,107.3
51,Tasiilaq,0,GL,1588641846,44,65.61,-37.64,3.00,22.3


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_weather.copy()
hotel_df = hotel_df.drop(["Cloudiness (%)", "Date", "Humidity (%)", "Max Temp (°C)", "Wind Speed (km-h)"], axis = 1)
hotel_df["Hotel Name"] = " "
hotel_df.reset_index(inplace = True, drop = 1)
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Dombarovskiy,RU,50.76,59.54,
1,Makkah al Mukarramah,SA,21.43,39.83,
2,Albany,US,42.60,-73.97,
3,Husavik,IS,66.04,-17.34,
4,Tasiilaq,GL,65.61,-37.64,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    try:
        coords = f"{row['Latitude']},{row['Longitude']}"
        hotel_params = {"location" : coords, "radius" : "5000", "type" : "hotel", "key" : api_key}
        hotel_response = requests.get(base_url, params = hotel_params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][1]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"


In [7]:
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Dombarovskiy,RU,50.76,59.54,"Center ""My Documents"""
1,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
2,Albany,US,42.60,-73.97,WGNA-FM Albany
3,Husavik,IS,66.04,-17.34,The Exploration Museum
4,Tasiilaq,GL,65.61,-37.64,Tasiilaq Heliport


In [8]:
#Keep the cities with actual hotels

my_hotels = hotel_df.loc[(hotel_df["Hotel Name"].str.contains("Hotel") |
             hotel_df["Hotel Name"].str.contains("Motel") |
             hotel_df["Hotel Name"].str.contains("Inn") |
             hotel_df["Hotel Name"].str.contains("Royal")
                         )]

print(len(my_hotels))
my_hotels

15


,City,Country,Latitude,Longitude,Hotel Name
1,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
10,Sterling,US,39.01,-77.43,Hampton Inn & Suites Washington-Dulles Interna...
11,Zambezi,ZM,-13.54,23.10,Royal Kutachika Lodge
18,Pacific Grove,US,36.62,-121.92,Lovers Point Inn
20,Haines Junction,CA,60.75,-137.51,Parkside Inn
21,Moree,AU,-29.47,149.85,Molika Springs Motel
23,Westport,US,41.14,-73.36,The Westport Inn
25,Saint Anthony,US,45.02,-93.22,Holiday Inn Express Roseville-St. Paul
26,Orange,US,33.70,-117.77,La Quinta Inn & Suites by Wyndham Irvine Spectrum
28,Pokaran,IN,26.92,71.92,Motel Pokaran - RTDC


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**hotel) for index, hotel in my_hotels.iterrows()]
hotel_locations = my_hotels[["Latitude", "Longitude"]]

#Create hotel markers
hotel_markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

#Add hotel markers to map
fig.add_layer(hotel_markers)

#Display again
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='5px', width='1000px'))

### --- Annexes ---

In [10]:
#TEST FOR HOTEL SEARCH IN CDMX
hotel_params = {
     "location" : "19.432608, -99.133209",
     "radius" : "5000",
     "type" : "hotel",
     "key" : api_key}

hotel_cdmx = requests.get(base_url, params = hotel_params).json()

In [11]:
#CITY INFO
print(hotel_cdmx["results"][0].keys())
hotel_cdmx["results"][0]

dict_keys(['geometry', 'icon', 'id', 'name', 'photos', 'place_id', 'reference', 'scope', 'types', 'vicinity'])


{'geometry': {'location': {'lat': 19.4326077, 'lng': -99.133208},
  'viewport': {'northeast': {'lat': 19.5927571, 'lng': -98.9604482},
   'southwest': {'lat': 19.1887101, 'lng': -99.3267771}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
 'id': '480f80745b1f911f3aed6a7301a1b75271d6cab4',
 'name': 'Mexico City',
 'photos': [{'height': 412,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108952706690871260850">Yihya Alali</a>'],
   'photo_reference': 'CmRaAAAAtQ7bHp6cD-qIm1_ZnsbLqEUL7vd7h0gsphxcRJfgoEOWtJl5y53biWewn_P01hVOOkuiMhupFjpsSMyFsM4Vl4QUhmXv1Y_0fq9r-LxTFQ4ypUYloziQRk_NJwXaAWX_EhC2o9HkvuMTDb88YonsraxqGhR9V5J-xjsqCjUXDSOVcKR8RZmfLQ',
   'width': 550}],
 'place_id': 'ChIJB3UJ2yYAzoURQeheJnYQBlQ',
 'reference': 'ChIJB3UJ2yYAzoURQeheJnYQBlQ',
 'scope': 'GOOGLE',
 'types': ['locality', 'political'],
 'vicinity': 'Mexico City'}

In [12]:
#FIRST HOTEL INFO
print(hotel_cdmx["results"][1].keys())
hotel_cdmx["results"][1]

dict_keys(['business_status', 'geometry', 'icon', 'id', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope', 'types', 'user_ratings_total', 'vicinity'])


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 19.4320832, 'lng': -99.1345917},
  'viewport': {'northeast': {'lat': 19.4333897302915,
    'lng': -99.13342471970849},
   'southwest': {'lat': 19.4306917697085, 'lng': -99.1361226802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
 'id': 'e96e035ea6e6f7da93d4834781242f4d5af6ca2a',
 'name': 'Gran Hotel Ciudad de México',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 5184,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103679786293097838208">Gran Hotel Ciudad de México</a>'],
   'photo_reference': 'CmRaAAAAyIAmELLTQWzDZk4OXOhp5V7vtfSWcVc66nwNXztCu15VuihahbNNOzRprGiMwCnrzzo7RYoV68K9u2g_zUPncF7waKkDcwnQ54KVqh3bNvpDRri7T-82_xAxmrmcjHcYEhARQp8LkktCVrROWQwhA0GAGhRhu1OlxEzJN6Qp79J1xN10xoSphg',
   'width': 3456}],
 'place_id': 'ChIJQRMdGVcBzoURTPsEkZCr5tY',
 'plus_code': {'compound_code': 'CVJ8+R5 Mexico City, Mexico',
  'global_code': '76F2CVJ8+R5'},
